In [1]:
import os
from glob import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import h5py

import math
import ephem
from datetime import datetime, time, timedelta
from math import pi, cos, sin
from scipy import optimize
from joblib import Parallel, delayed
from mpl_toolkits.basemap import Basemap
# import xesmf as xr

from diurnal_utils import *
%run diurnal_utils.py

In [6]:
out_folder_identifier = 'bin_pt2_hr_15_yr_round_2_mode_year_mean_bootstrap_5yr_final' #label to append to output path

start_date = '2000-06'
# end_date = '2000-07'
end_date =  '2016-06'


In [5]:
# for sampo
# save_output_dir = '/export/data1/cchristo/diurnal_analysis_results/GPM_' + out_folder_identifier + '/'

# for tofu
# save_output_dir = '/net/fluo/data2/groupMembers/cchristo/gpm_results/' + out_folder_identifier + '/'

# for curry
save_output_dir = '/net/fluo/data2/groupMembers/cchristo/gpm_results/' + out_folder_identifier + '/'



# save_figs_dir = '/home/cchristo/proj_tapio/figs/diurnal_cycle_figs/gpm_2000_2010_precip/'

In [7]:

# for sampo
# model_dir = '/export/data1/cchristo/gpm_data/gpmdata/'
# model_dir = '/export/data1/cchristo/gpm_data/gpmdata_regridded_gfdl_cm4/'
# # model_dir = '/export/data1/cchristo/gpm_data/gpmdata_regridded_grid1/'

# for tofu
# model_dir = '/net/fluo/data2/groupMembers/cchristo/gpm_grid1/'
model_dir = '/net/fluo/data2/groupMembers/cchristo/gpm_cm4/'


years_to_include = range(2000, 2017)
# years_to_include = range(2000, 2001)
years_to_include = [str(year_ii) for year_ii in years_to_include]

result = [y for x in os.walk(model_dir) for y in glob(os.path.join(x[0], '*.nc'))]


# filter paths based on year
result_filt =[]
for path_i in result: 
    if (os.path.dirname(path_i).split('/')[-4] in years_to_include):
        result_filt.append(path_i)

# get only certain years
# result = sorted(result)
result = sorted(result_filt)

In [6]:
# print(result[-1])

/net/fluo/data2/groupMembers/cchristo/gpm/2016/12/31/imerg/3B-HHR.MS.MRG.3IMERG.20161231-S233000-E235959.1410.V06B.nc


In [8]:
# %%time
ds = xr.open_mfdataset(result, combine='nested', concat_dim = 'time')
# ds = xr.open_dataset(result[0],  engine = 'h5netcdf')

In [82]:
# ds['lon'] = ds['lon'].where(ds['lon'] > 0, ds['lon'] + 360)

In [9]:
ds = ds.sel(time = slice(start_date, end_date))
ds = ds.sel(lat= slice(-60, 60))
# ds.load()
# print('Dataset Loaded')

In [10]:
# ds['precipitationCal'].fillna(0)


In [ ]:
# ds_mean = ds['precipitationCal'].mean(dim = 'time')
# ds_mean.load()

# ds_max = ds['precipitationCal'].mean(dim = 'time')
# ds_max.load()


In [ ]:
# ds_mean.plot.imshow(vmin =0, vmax = 1, cmap = plt.get_cmap('gist_ncar'))
# ds_max.plot.imshow(vmin =0, vmax = 0.25 , cmap = plt.get_cmap('gist_ncar'))

In [ ]:
# ds_mean.plot.imshow(vmin =0, vmax = 2, cmap = plt.get_cmap('gist_ncar'))

In [ ]:
# ds.isel(time = 0)['precipitationCal'].plot.imshow(vmin =0, vmax = 10)

In [58]:
# ds['precipitationCal']
# len(ds['time'])
%run diurnal_utils.py

ds = xr.open_mfdataset(result, combine='nested', concat_dim = 'time')

ds = ds.sel(time = slice(start_date, end_date))
ds = ds.sel(lat= slice(-60, 60))

out_ds, out_ds_means = diurnal_analysis_2_mode_full_bootstrapped(ds, 
                                    field_id = 'precipitationCal', 
                                    grid_time_resolution_hours = 0.5,
                                    time_resolution_hours = 0.2,
                                    samplesize = 48*365*5,
                                    blocksize = 48,
                                    nboot = 200)

  0%|          | 0/1440 [00:00<?, ?it/s]

0


100%|██████████| 1440/1440 [00:03<00:00, 364.00it/s]
/home/cchristo/proj_tapio/diurnal_research/diurnal_utils.py:453: RuntimeWarning: Mean of empty slice
  f_bar_k = np.nanmean(masked_field, axis = 0)
  0%|          | 0/120 [00:00<?, ?it/s]

Performing Cos Fit


 28%|██▊       | 33/120 [00:10<00:26,  3.33it/s]/home/cchristo/.conda/envs/diurnal_research/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 120/120 [00:36<00:00,  3.31it/s]

Finished Cos Fit


In [59]:
res

(<xarray.Dataset>
 Dimensions:       (bootstrap_id: 1, lat: 120, lon: 288)
 Coordinates:
   * bootstrap_id  (bootstrap_id) int64 0
   * lon           (lon) float64 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
   * lat           (lat) float64 -59.5 -58.5 -57.5 -56.5 ... 56.5 57.5 58.5 59.5
 Data variables:
     mu_season     (bootstrap_id, lat, lon) float64 0.07827 0.05077 ... 0.05744
     ampl_season   (bootstrap_id, lat, lon) float64 0.05305 0.02147 ... 0.07484
     phase_season  (bootstrap_id, lat, lon) float64 13.09 8.859 ... 19.96 22.28,
 <xarray.Dataset>
 Dimensions:                (lat: 120, lon: 288, season: 1, time: 48)
 Coordinates:
   * season                 (season) int64 0
   * time                   (time) float64 0.5 1.0 1.5 2.0 ... 23.0 23.5 24.0
   * lon                    (lon) float64 0.625 1.875 3.125 ... 356.9 358.1 359.4
   * lat                    (lat) float64 -59.5 -58.5 -57.5 ... 57.5 58.5 59.5
 Data variables:
     precipitationCal_mean  (season, time, lat, 

In [57]:
# res[0].isel(season = 2)['phase_season'].plot.imshow(cmap = plt.get_cmap('twilight_shifted'))

In [22]:

# out_ds, out_ds_means = diurnal_analysis_2_mode_seasonal(ds, 
#                                         field_id = 'precipitationCal', 
#                                         grid_time_resolution_hours = 0.5,
#                                         time_resolution_hours = 0.2)
#                                         time_resolution_hours = 0.2) #0.5)
# ds = xr.open_mfdataset(result, combine='nested', concat_dim = 'time')

# ds = ds.sel(time = slice(start_date, end_date))
# ds = ds.sel(lat= slice(-60, 60))

out_ds, out_ds_means = diurnal_analysis_2_mode_full(ds, 
                                        field_id = 'precipitationCal', 
                                        grid_time_resolution_hours = 0.5,
                                        time_resolution_hours = 0.2)

if not os.path.exists(save_output_dir):
    os.makedirs(save_output_dir)

out_ds.to_netcdf(save_output_dir + start_date + '_' + end_date + '_precip.nc')
out_ds_means.to_netcdf(save_output_dir + start_date + '_' + end_date + '_precip_diurnal_means.nc')


JJA


100%|██████████| 2928/2928 [00:05<00:00, 512.10it/s]
/home/cchristo/proj_tapio/diurnal_research/diurnal_utils.py:361: RuntimeWarning: Mean of empty slice
  f_bar_k = np.nanmean(masked_field, axis = 0)
  0%|          | 0/120 [00:00<?, ?it/s]

Performing Cos Fit


 29%|██▉       | 35/120 [00:10<00:25,  3.38it/s]/home/cchristo/.conda/envs/diurnal_research/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
100%|██████████| 120/120 [00:35<00:00,  3.37it/s]


New!
Finished Cos Fit


In [24]:
if not os.path.exists(save_output_dir):
    os.makedirs(save_output_dir)

out_ds.to_netcdf(save_output_dir + start_date + '_' + end_date + '_precip.nc')
out_ds_means.to_netcdf(save_output_dir + start_date + '_' + end_date + '_precip_diurnal_means.nc')

In [ ]:
# make_four_panel(mu_season , 
# #                 cmap = plt.get_cmap('bwr'),
#                 cmap = plt.get_cmap('gist_ncar'),
#                 lats = ds['lat'].values, 
#                 lons = ds['lon'].values,
#                 vmin = 0,
#                 vmax = 0.8,
#                 title = r'$\mu$',
# #                 axis = plt.axis([220, 300, 10, 50]), 
#                 save_fig_path= save_figs_dir + 'GFDL_CM4_means_pr.png')

In [ ]:
# make_four_panel(sigma_season , 
#                 lats = ds['lat'].values, 
#                 lons = ds['lon'].values,
#                 vmin = 0, vmax = 0.2, 
# #                 cmap = plt.get_cmap('bwr'),
#                 cmap = plt.get_cmap('gist_ncar'),
#                 title = r'$\sigma$',
# #                 axis = plt.axis([220, 300, 10, 50]), 
#                 save_fig_path= save_figs_dir + 'GFDL_CM4_stds_pr.png')

In [ ]:
# make_four_panel(ampl_season , 
#                 lats = ds['lat'].values, 
#                 lons = ds['lon'].values,
# #                 vmax = 0.00003, 
# #                 cmap = plt.get_cmap('bwr'),
#                 cmap = plt.get_cmap('gist_ncar'),
#                 title = r'$A$',
#                 vmin = 0, vmax = 0.2,
# #                 axis = plt.axis([220, 300, 10, 50]), 
#                 save_fig_path= save_figs_dir + 'GFDL_CM4_ampl_pr.png')

In [ ]:
# make_four_panel(phase_season , 
#                 lats = ds['lat'].values, 
#                 lons = ds['lon'].values,
# #                 vmin = 0, vmax = 24, 
#                 cmap = plt.get_cmap('twilight'),
#                 title = r'$\Phi$',
# #                 axis = plt.axis([220, 300, 10, 50]), 
#                 save_fig_path= save_figs_dir + 'GFDL_CM4_phase_pr.png')

In [18]:
# out_ds = xr.Dataset()
# out_ds['mu_season'] = make_da_from_dict(mu_season, ds)
# out_ds['sigma_season'] = make_da_from_dict(sigma_season, ds)
# out_ds['ampl_season'] = make_da_from_dict(ampl_season, ds)
# out_ds['phase_season'] = make_da_from_dict(phase_season, ds)

In [22]:
# out_ds.to_netcdf(save_output_dir + 'gpm_2000_2010_precip.nc')